In [1]:
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sio
from numpy import random
from scipy.stats import norm
import pickle
import scipy

In [ ]:
# functions to calculate all individual-unit metrics
def tuning_calculate(firingdata,angle_list):
    divisions = 50
    dtheta = 1/divisions*360
    theta = np.arange(0,divisions)/divisions*360
    tuning_fr = np.zeros_like(theta)
    for i in np.arange(0,divisions):
        idx = (angle_list>i*dtheta)&(angle_list<(i+1)*dtheta)
        tuning_fr[i] = np.mean(firingdata[idx]) 
    return tuning_fr
def frall_calculate(neuraldata,angle_list_rnnout,angle_list_target):
    [cellnum,timet] = np.shape(neuraldata)
    tuningfr_all_rnn = np.zeros([100,50])
    tuningfr_all_target = np.zeros([100,50])
    for celln in np.arange(cellnum):
        # print(celln)
        fr = neuraldata[celln,:]
        tuningfr_rnn = tuning_calculate(fr,angle_list_rnnout)
        # plt.plot(tuningfr_rnn)
        tuningfr_all_rnn[celln,:] = tuningfr_rnn
        tuningfr_target = tuning_calculate(fr,angle_list_target)
        tuningfr_all_target[celln,:] = tuningfr_target
    return tuningfr_all_rnn,tuningfr_all_target
def hd_tuning_permutation(neuralfiring,anglelist):
    divisions = 50
    permutationtimes = 50
    dtheta = 1/divisions*360
    tuning_fr = np.zeros([permutationtimes,divisions],dtype=float)
    tuning_fr_shuffle = np.zeros([permutationtimes,divisions],dtype=float)
    neuronT = len(neuralfiring)
    # do tuning permu
    for i in np.arange(permutationtimes):
        alltimepoints = [i for i in np.arange(neuronT)]
        temphalf = np.floor(neuronT/5)
        temphalf = temphalf.astype(int)
        half_chosen_t = random.choice(alltimepoints,temphalf, replace=False)
        half_chosen_angle = anglelist[half_chosen_t]
        neural_fr_new = neuralfiring[half_chosen_t]
        for j in np.arange(0,divisions):
            idx = (half_chosen_angle>j*dtheta)&(half_chosen_angle<(j+1)*dtheta)
            tuning_fr[i,j] = np.mean(neural_fr_new[idx]) 
        # do baseline permu
        shuffangle= random.permutation(half_chosen_angle)
        for j in np.arange(0,divisions):
            idx = (shuffangle>j*dtheta)&(shuffangle<(j+1)*dtheta)
            tuning_fr_shuffle[i,j] = np.mean(neural_fr_new[idx]) 
    tuning_permu_mean = np.mean(tuning_fr,axis=0)
    tuning_permu_std = np.std(tuning_fr,axis=0)
    tuning_shuffle_mean = np.mean(tuning_fr_shuffle,axis=0)
    tuning_shuffle_std = np.std(tuning_fr_shuffle,axis=0)
    # p value of tuning for each division
    z = (tuning_permu_mean-tuning_shuffle_mean)/tuning_shuffle_std
    p_value= 1-norm.cdf(z)
    return tuning_permu_mean,tuning_permu_std,p_value
def mean_vector_length(tuning_fr):
    divisions = 50
    theta = np.arange(0,divisions)/divisions*360 
    mvl = sum(tuning_fr[i]*np.exp(1j*theta[i]*np.pi/180) for i in range(len(tuning_fr)))/sum(tuning_fr)
    mvl = np.abs(mvl)
    return mvl
def chance_vector_length(neuralfiring, anglelist):
    chance_mvl = []
    dt_rnn = 0.025
    min_shift_points = np.round(30/dt_rnn) # shift at least 30 seconds
    lengtht = len(neuralfiring)
    for i in np.arange(100):
        shift = np.random.randint(lengtht-min_shift_points)+min_shift_points
        # print(shift)
        anglelist_shift = np.roll(anglelist, int(shift))
        tuning_fr = tuning_calculate(neuralfiring,anglelist_shift)
        mvl = mean_vector_length(tuning_fr)
        chance_mvl.append(mvl)
    return chance_mvl
def hd_tuning_mutual_info_calculate(neuralfiring,anglelist,mean_fr):
    tuning_fr = tuning_calculate(neuralfiring,anglelist)
    divisions = 50
    dtheta = 1/divisions*360
    theta = np.arange(0,divisions)/divisions*360
    pbin = np.zeros_like(theta)
    for i in np.arange(0,divisions):
        idx = (anglelist>i*dtheta)&(anglelist<(i+1)*dtheta)
        pbin[i] = np.sum(idx)/len(anglelist)
    zerosindx = np.where(tuning_fr < 0.00001) # remove bins with firing rate too small, otherwise the mutual information will be -inf
    if len(zerosindx[0])!=0:
        pbin = np.delete(pbin,zerosindx)
        tuning_fr = np.delete(tuning_fr,zerosindx)
    if mean_fr > 0.00001:
        mi = np.sum(pbin*tuning_fr*np.log2(tuning_fr/mean_fr))
    else:
        mi = np.nan
    return mi
def p_value_hd_tuning_mi_calculate(neuralfiring,anglelist):
    mi_observe = hd_tuning_mutual_info_calculate(neuralfiring,anglelist, np.mean(neuralfiring))
    # print('mean_fr = ', np.mean(neuralfiring))
    MI_all = np.array([],dtype=float)
    dt_bin = 0.025
    min_shift_points = np.round(30/dt_bin) # shift at least 30 seconds
    lengtht = len(neuralfiring)
    for i in np.arange(100):
        shift = np.random.randint(lengtht-min_shift_points)+min_shift_points
        # print(shift)
        anglelist_shift = np.roll(anglelist, int(shift))
        tuning_fr = tuning_calculate(neuralfiring,anglelist_shift)
        mi = hd_tuning_mutual_info_calculate(neuralfiring,anglelist, np.mean(neuralfiring))
        MI_all = np.append(MI_all,mi)
    p_value_mi = np.sum(MI_all>mi_observe)/1000
    return p_value_mi

def v_fr_partial_correlation_calulate(firingdata,velocity,angle):
    firingdata = scipy.stats.zscore(firingdata)
    indx_pos = np.where(velocity>=0)[0]
    firingdata_pos = firingdata[indx_pos]
    ahv_fr_cor_pos = np.corrcoef(velocity[indx_pos],firingdata_pos)[0,1]
    angle_fr_cor_pos = np.corrcoef(angle[indx_pos],firingdata_pos)[0,1]
    ahv_angle_cor_pos = np.corrcoef(velocity[indx_pos],angle[indx_pos])[0,1]
    ahv_fr_partial_pos = (ahv_fr_cor_pos-angle_fr_cor_pos*ahv_angle_cor_pos)/np.sqrt((1-angle_fr_cor_pos**2)*(1- ahv_angle_cor_pos**2))
    # index for velocity<0
    indx_neg = np.where(velocity<0)[0]
    firingdata_neg = firingdata[indx_neg]
    ahv_fr_cor_neg = np.corrcoef(velocity[indx_neg],firingdata_neg)[0,1]
    angle_fr_cor_neg = np.corrcoef(angle[indx_neg],firingdata_neg)[0,1]
    ahv_angle_cor_neg = np.corrcoef(velocity[indx_neg],angle[indx_neg])[0,1]
    ahv_fr_partial_neg = (ahv_fr_cor_neg-angle_fr_cor_neg*ahv_angle_cor_neg)/np.sqrt((1-angle_fr_cor_neg**2)*(1- ahv_angle_cor_neg**2))
    ahv_fr_cor = np.max(np.abs([ahv_fr_cor_pos,ahv_fr_cor_neg]))
    ahv_fr_partial_cor = np.max(np.abs([ahv_fr_partial_pos,ahv_fr_partial_neg]))
    return ahv_fr_cor,ahv_fr_partial_cor

def v_tuning_calculate(firingdata,velocity):
    delta_v = 0.1
    v_min = np.min(velocity)
    v_max = np.max(velocity)
    v_min = np.floor(v_min/delta_v)*delta_v
    v_max = np.ceil(v_max/delta_v)*delta_v
    angular_v = np.arange(v_min,v_max+delta_v,delta_v)
    tuning_fr = np.zeros_like(angular_v)
    for i in np.arange(len(tuning_fr)-1):
        idx = (velocity>angular_v[i])&(velocity<angular_v[i+1])
        intemp = np.where(idx==1)
        # print([angular_v[i], angular_v[i+1], intemp])
        if len(firingdata[idx]) != 0:
            tuning_fr[i] = np.mean(firingdata[idx]) 
        else:
            tuning_fr[i] = np.nan
    # print(angular_v)
    return angular_v, tuning_fr   

def ahv_permutation_tuning(neuralfiring,velocity):
    permutationtimes = 50
    delta_v = 0.1
    range_to_show = [-3,3]
    v_min = range_to_show[0]
    v_max = range_to_show[1]
    v_min = np.floor(v_min/delta_v)*delta_v
    v_max = np.ceil(v_max/delta_v)*delta_v
    angular_v = np.arange(v_min,v_max,delta_v)
    # theta = np.arange(0,divisions)/divisions*2*np.pi
    tuning_fr_permu = np.zeros([permutationtimes,len(angular_v)],dtype=float)
    neuronT = len(neuralfiring)
    # do tuning permu
    for i in np.arange(permutationtimes):
        alltimepoints = [i for i in np.arange(neuronT)]
        temphalf = np.floor(neuronT/5)
        temphalf = temphalf.astype(int)
        half_chosen_t = random.choice(alltimepoints,temphalf, replace=False)
        half_chosen_angle = velocity[half_chosen_t]
        neural_fr_new = neuralfiring[half_chosen_t]
        for j in np.arange(len(angular_v)-1):
            idx = (half_chosen_angle>angular_v[j])&(half_chosen_angle<angular_v[j+1])
            if len(neural_fr_new[idx]) != 0:
                tuning_fr_permu[i,j] = np.mean(neural_fr_new[idx]) 
            else:
                tuning_fr_permu[i,j] = np.nan
    tuning_permu_mean = np.nanmean(tuning_fr_permu,axis=0)
    tuning_permu_std = np.nanstd(tuning_fr_permu,axis=0)
    return tuning_permu_mean,tuning_permu_std
def fwhm_calculate(tuning_curve):
    data = tuning_curve-np.min(tuning_curve)
    max_indx = np.argmax(data)
    max_value = np.max(data)
    if max_indx >0:
        for i in np.arange(max_indx):
            valuehere = data[max_indx-i-1]
            if valuehere < max_value/2:
                break
        lefthalf_indx = max_indx-i-1
        # print(lefthalf_indx,valuehere,max_value/2)
        # print(lefthalf_indx-1,data[lefthalf_indx-1])
        # print(lefthalf_indx+1,data[lefthalf_indx+1])
    else:
        lefthalf_indx = 0
    if len(data)-max_indx-1>0:
        for i in np.arange(len(data)-max_indx):
            valuehere = data[max_indx+i]
            if valuehere < max_value/2:
                break
        righthalf_indx = max_indx+i
    else:
        righthalf_indx = len(data)-1

    if lefthalf_indx == 0 and righthalf_indx!=len(data)-1:
        lefthalf_indx = righthalf_indx
        for i in np.arange(len(data)-1):
            valuehere = data[len(data)-i-1]
            if valuehere < max_value/2:
                break
        if i == 0: 
            righthalf_indx = len(data)-1
        else:
            righthalf_indx = len(data)-i-1
        half_peak_width =len(data)-(righthalf_indx-lefthalf_indx)
        left_half_real = righthalf_indx
        right_half_real = lefthalf_indx
    elif lefthalf_indx != 0 and righthalf_indx==len(data)-1:
        righthalf_indx = lefthalf_indx
        for i in np.arange(len(data)-1):
            valuehere = data[i]
            if valuehere < max_value/2:
                break
        lefthalf_indx = i
        half_peak_width =len(data)-(righthalf_indx-lefthalf_indx)
        left_half_real = righthalf_indx
        right_half_real = lefthalf_indx
    else:
        # lefthalf_indx = lefthalf_indx
        # righthalf_indx = righthalf_indx
        half_peak_width = righthalf_indx-lefthalf_indx
        left_half_real = lefthalf_indx
        right_half_real = righthalf_indx
    return half_peak_width,left_half_real,right_half_real

In [ ]:
# load data
varname = '../../simulated_data/epoch_test/output_simulated_200trials_1000t_epoch_'+str(500)+'_0228.mat' 
data = sio.loadmat(varname)
data1 = data['data']
data1 = np.array(data1)
neuraldata = data1[0:100,:]
angle_list_target = data1[-3,:]
angle_list_rnnout = data1[-2,:]
angular_velocity = data1[-1,:]

In [ ]:
# network performance and unit firing visualization (Fig. 2)
tuningfr_all_rnn,tuningfr_all_target = frall_calculate(neuraldata,angle_list_rnnout,angle_list_target)
bf = np.argmax(tuningfr_all_rnn,axis=1)
re_order = np.argsort(bf)
angle_list_rnnout = np.mod(angle_list_rnnout,360)
angle_list_target = np.mod(angle_list_target,360)
pointplot = 1000
t = np.arange(0,pointplot,1)*0.025 # 0.025s per step
# color code for different values of neural activity
neuraldata = data1[re_order,0:pointplot]
y_min = np.min(np.min(neuraldata)) 
y_max = np.max(np.max(neuraldata))
cmap = plt.cm.Greens
norm = plt.Normalize(neuraldata.min(), neuraldata.max())  
colorcode = cmap(norm(neuraldata)) 
fig = plt.figure(figsize=(20,10))
for i in range(100):
    _ = plt.scatter(t, np.ones_like(t)*i, c=colorcode[i], alpha=0.3, s=3)
    if i == 1:
        plt.colorbar(_, label='neural activity')
# plot y axis to the right
plt.twinx()  
plt.plot(t,angle_list_target[0:pointplot],label='target',linewidth=5,color='black')
plt.plot(t,angle_list_rnnout[0:pointplot],label='rnnout',linewidth=5,color='red',linestyle='--')
fig.savefig('../../figures/RNN_performance_and_network_activities.pdf', bbox_inches='tight', transparent=True, format='pdf')  

In [ ]:
# metric calculation
angular_velocity = angular_velocity*40 # convert to rad/s
hd_mi_all = []
hd_mi_p_value_all = []
ahv_fr_cor_all = []
mean_fr_all = []
mvl_all = []
p_mvl_all = []
p_value_hd_tuning_all = []
ahv_fr_partial_cor_all = []
ahv_fr_cor_all = []
divisions = 50
hd_tuning_plot_all = np.zeros([divisions*3,100],dtype=float)
p_value_hd_tuning_all= np.zeros([divisions,100],dtype=float)
theta = np.arange(0,divisions)/divisions*360
ahv_tuning_plot_all = np.zeros([61,100],dtype=float)
half_peak_width_all = np.zeros(100)
for i in np.arange(100):
    print(i)
    neuralfiring = neuraldata[i,:]
    tuning_fr= tuning_calculate(neuralfiring,angle_list_target)
    # random sample HD tuning permutation
    tuning_permu_mean,tuning_permu_std,p_hd_tuning_value = hd_tuning_permutation(neuralfiring,angle_list_target)
    hd_tuning_plot_all[:,i] = tuningplotdata
    p_value_hd_tuning_all.append(p_hd_tuning_value)
    tuningplotdata = np.concatenate((tuning_fr,tuning_permu_mean,tuning_permu_std),axis=0)
    # tuning width
    half_peak_width_i,left_half_index,right_half_index = fwhm_calculate(tuning_fr)
    half_peak_width_all[i] = half_peak_width_i
    # mvl calculaton
    mvl = mean_vector_length(tuning_fr)
    chance_mvl = chance_vector_length(neuralfiring,angle_list_target)
    p_mvl = np.sum(chance_mvl>mvl)/len(chance_mvl)
    mvl_all.append(mvl)
    p_mvl_all = np.append(p_mvl_all,p_mvl)
    ahv_fr_cor,ahv_fr_partial_cor = v_fr_partial_correlation_calulate(neuralfiring,angular_velocity,angle_list_target)
    ahv_fr_partial_cor_all = np.append(ahv_fr_partial_cor_all,ahv_fr_partial_cor)
    ahv_fr_cor_all = np.append(ahv_fr_cor_all,ahv_fr_cor)
    v_range, tuning_fr = v_tuning_calculate(neuralfiring,angular_velocity)
    tuning_permu_mean, tuning_permu_std = ahv_permutation_tuning(neuralfiring,angular_velocity)
    tuningplotdata = np.concatenate((tuning_fr,tuning_permu_mean,tuning_permu_std),axis=0)
    ahv_tuning_plot_all[:,i] = tuningplotdata
    
    mean_fr = np.mean(neuraldata[i,:])
    mean_fr_all.append(mean_fr)
    if mean_fr>0.0001:
        p_value_mi = p_value_hd_tuning_mi_calculate(neuraldata[i,:],angle_list_target)
        print(p_value_mi)
        hd_mi_p_value_all.append(p_value_mi)
        mi_neuron = hd_tuning_mutual_info_calculate(neuraldata[i,:],angle_list_target,np.mean(neuraldata[i,:]))
        hd_mi_all.append(mi_neuron)
    else:
        hd_mi_p_value_all.append(np.nan)
        hd_mi_all.append(np.nan)
data = {'mvl_all':mvl_all,
        'hd_tuning_plot_all': hd_tuning_plot_all,
        'ahv_tuning_plot_all': ahv_tuning_plot_all,
        'p_value_hd_tuning_all': p_value_hd_tuning_all,
        'hd_mi_all': hd_mi_all,
        'hd_mi_p_value_all': hd_mi_p_value_all,
        'ahv_fr_cor_all': ahv_fr_cor_all,
        'ahv_fr_partial_co_all': ahv_fr_partial_cor_all,
        'half_peak_width_all': half_peak_width_all}
with open('../../../results/All_metrics_individual_units_epoch500.pickle', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
# plot HD tuning curve of all units for example simulated session
color1 = [254/255,129/255,126/255]
color2 = [129/255,184/255,223/255]
panels = 10
non_singlelist = [54,67,84,13,4,6,88,20,97,35,83] # first determined by eye
non_selective_list = [12,42,44,45,51,68,69,71,79,82,94,95]
singlelist = np.array(list(set(np.arange(100))-set(non_singlelist)-set(non_selective_list)))
non_singlelist = np.array(non_singlelist)
singlelist = singlelist.astype(int)
non_singlelist = non_singlelist.astype(int)
with open('../../../results/All_metrics_individual_units_epoch500.pickle', 'rb') as f:
    data = pickle.load(f)
    tuning_plot_all = data['hd_tuning_plot_all']
divisions = 50
theta = np.arange(0,divisions)/divisions*36
fig, axs = plt.subplots(panels, panels,figsize=(20,20))
for ploti in np.arange(100):
    tuning_fr = tuning_plot_all[0:50,i]
    tuning_permu_mean = tuning_plot_all[50:100,ploti]
    tuning_permu_std = tuning_plot_all[100:150,ploti]
    xd = ploti//panels
    yd = ploti-(ploti//panels)*panels
    maxtun = np.max(tuning_permu_mean+tuning_permu_std)
    mintun = np.min(tuning_permu_mean-tuning_permu_std)
    if ploti in non_selective_list:
        _ = axs[xd,yd].plot(theta,tuning_fr,color='k',alpha=0.5)
        _ = axs[xd,yd].fill_between(theta,tuning_permu_mean-tuning_permu_std,tuning_permu_mean+tuning_permu_std,alpha=0.2, color= 'k')
    elif ploti in singlelist and ploti not in non_selective_list:
        _ = axs[xd,yd].plot(theta,tuning_fr,color= color2)
        _ = axs[xd,yd].fill_between(theta,tuning_permu_mean-tuning_permu_std,tuning_permu_mean+tuning_permu_std,alpha=0.5, color= color2)
    elif ploti not in singlelist and ploti not in non_selective_list:
        _ = axs[xd,yd].plot(theta,tuning_fr,color= color1)
        _ = axs[xd,yd].fill_between(theta,tuning_permu_mean-tuning_permu_std,tuning_permu_mean+tuning_permu_std,alpha=0.5, color= color1)
        titlename = "%s"% (ploti)+', %.3f' % hd_mi_all[ploti]
    _ = axs[xd,yd].set_title(titlename)
    _ = axs[xd,yd].set_xticks([])
    _ = axs[xd,yd].set_yticks([])
fig.savefig('../../figures/HD_tuning_example_RNN100.pdf', bbox_inches='tight', transparent=True, format='pdf')

# plot AHV tuning for all units
with open('../../../results/All_metrics_individual_units_epoch500.pickle', 'rb') as f:
    data = pickle.load(f)
    tuning_plot_all = data['ahv_tuning_plot_all']
    ahv_fr_partial_cor = data['ahv_fr_partial_cor']
delta_v = 0.1
v_range = np.arange(-3,3,delta_v)
divisions = len(v_range)
for ploti in np.arange(100):
    tuning_fr = tuning_plot_all[0:50,i]
    tuning_permu_mean = tuning_plot_all[50:100,i]
    tuning_permu_std = tuning_plot_all[100:150,i]
    xd = ploti//10
    yd = ploti-(ploti//10)*10
    if ploti in singlelist and ploti not in non_selective_list:
        _ = axs[xd, yd].plot(v_range, tuning_fr,color= color2)
        _ = axs[xd, yd].fill_between(v_range,tuning_permu_mean-tuning_permu_std,tuning_permu_mean+tuning_permu_std,alpha=0.5, color= color2)
    elif ploti in non_singlelist and ploti not in non_selective_list:
        _ = axs[xd, yd].plot(v_range,tuning_fr,color= color1)
        _ = axs[xd, yd].fill_between(v_range,tuning_permu_mean-tuning_permu_std,tuning_permu_mean+tuning_permu_std,alpha=0.5, color= color1)
    else:
        _ = axs[xd, yd].plot(tuning_fr,color='k',alpha=0.5)
    titlename = '%.3f' % ahv_fr_partial_cor[ploti]
    _ = axs[xd, yd].set_title(titlename)
    _ = axs[xd,yd].set_xticks([])
    _ = axs[xd,yd].set_yticks([])
    # _ = plt.axis('off')

fig.savefig('../../figures/AHV_tuning_all_neurons_RNN_epo500.pdf', bbox_inches='tight', transparent=True, format='pdf')    

In [ ]:
# spike width vs mi and partial ahv-fr (Fig. 5A)
from scipy.stats import pearsonr
with open('../../../results/All_metrics_individual_units_epoch500.pickle', 'rb') as f:
    data = pickle.load(f)
ahv_fr_partial_cor_all = np.array(data['ahv_fr_partial_cor_all'])
hd_mi_all = np.array(data['hd_mi_all'])
half_peak_width_all = np.array(data['half_peak_width_all'])

y = half_peak_width_all[singlelist]
x = hd_mi_all[singlelist]
# z = np.abs(ahv_fr_cor_all[singlelist])
z = np.abs(ahv_fr_partial_cor_all[singlelist])

r, p = pearsonr(y, x)
print(r,p)
# print(p)
r = np.corrcoef(y, x)[0, 1]

# Calculate slope and intercept
slope = r * np.std(x) / np.std(y)
intercept = np.mean(x) - slope * np.mean(y)

# Plot regression line
fig = plt.figure()
_ = ax = plt.subplot(111)
ax.scatter(y,x,marker='^',s = 80,edgecolors = color2,facecolors = 'none')

y_indx = np.argsort(y)
y_inc = y[y_indx]
ax.plot(y_inc, slope*y_inc + intercept, color= color2, linestyle='--')
plt.xlabel('Half peak width')
plt.ylabel('HD MI')

# plty z,y in right axis
ax2 = plt.gca().twinx()
ax2.scatter(y,z,marker='o',s = 80,edgecolors = color2, facecolors = 'none')
ax2.set_ylabel('AHV FR correlation ')
r,p = pearsonr(y,z)
print(r,p)
r = np.corrcoef(y,z)[0,1]
slope = r*np.std(z)/np.std(y)
intercept = np.mean(z)-slope*np.mean(y)
y_indx = np.argsort(y)
y_inc = y[y_indx]
plt.plot(y_inc, slope*y_inc + intercept, color= color2, linestyle='--')
tickpos = [7.5,10,12.5,15,17.5,20,22.5]
ax2.set_xticks(tickpos)
peak_width_tick = np.array(tickpos)/50*2*np.pi
ax2.set_xticklabels(['{:.1f}'.format(xi) for xi in peak_width_tick])
plt.ylim(-0.1,0.8)
fig.savefig('../../figures/spike_width_VS_HD_AHV_partial_cor_unimodal.pdf', bbox_inches='tight', transparent=True, format='pdf')  

fig = plt.figure(figsize=(0.8,4))
hd_multimodal = hd_mi_all[non_singlelist]
ahv_fr_multimodal = np.abs(ahv_fr_cor_all[non_singlelist])

x = [1]*len(hd_multimodal)
plt.scatter(x, hd_multimodal,marker='^',s=80,edgecolors = color1, facecolors = 'none')
plt.ylim(0,0.4)
plt.ylabel('HD MI')
ax2 = plt.gca().twinx()
x = [2]*len(ahv_fr_multimodal)
plt.scatter(x, ahv_fr_multimodal, marker='o',s=80,edgecolors = color1, facecolors = 'none')
plt.ylim(-0.1,0.8)
plt.xlim(0,3)
fig.savefig('../../../figures/spike_width_VS_HD_AHV_partial_cor_multimodal.pdf', bbox_inches='tight', transparent=True, format='pdf')  
